# 06-build-model-TextCatEnsemble

## Main objectives:

- Use spaCy's TextCatEnsemble to train a stacked ensemble of a linear bag-of-words model and a neural network model.
    - [https://spacy.io/api/architectures#TextCatEnsemble](https://spacy.io/api/architectures#TextCatEnsemble)
- Use spaCy generated docs to train this model
- Run basic validation and evaluation 

In [1]:
import spacy
# load an english language model in spacy
nlp = spacy.load("en_core_web_lg")

In [2]:
!ls data

dev.spacy  train.spacy


# Validate configuration file

In [3]:
# validate configuration
!python -m spacy debug config ./config/config-TextCatEnsemble.cfg


============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ Config is valid


In [4]:
!python -m spacy debug data ./config/config-TextCatEnsemble.cfg


============================ Data file validation ============================
✔ Corpus is loadable
✔ Pipeline can be initialized with data

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, textcat
5000 training docs
5000 evaluation docs
✔ No overlap between training and evaluation data

============================== Vocab & Vectors ==============================
ℹ 282161 total word(s) in the data (18163 unique)
ℹ No word vectors present in the package

================================== Summary ==================================
✔ 3 checks passed


# Train the model

In [ ]:
!python -m spacy train ./config/config-TextCatEnsemble.cfg --output ./models/textCatEnsemble

ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-03-22 22:42:33,917] [INFO] Set up nlp object from config
[2021-03-22 22:42:34,439] [INFO] Pipeline: ['tok2vec', 'textcat']
[2021-03-22 22:42:34,459] [INFO] Created vocabulary
[2021-03-22 22:42:34,459] [INFO] Finished initializing nlp object
[2021-03-22 22:42:49,172] [INFO] Initialized pipeline components: ['tok2vec', 'textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.23
wandb: Syncing run true-bird-19
wandb: ⭐️ View project at https://wandb.ai/garza/yelp-polarity
wandb: 🚀 View run at https://wandb.ai/garza/yelp-polarity/runs/31v8ob3l
wandb: Run data is saved locally in /home/garza/notebooks/aicamp/yelp-project/wandb/run-20210322_224254-31v8ob3l
w

# Evaluate our best model output and save metrics to disk

For hyperparameter tuning, experimented with different BOW attributes and TexCatEnsemble parameters:

- ngram_size = 4 (TODO: should try again with ngram_size = 2 after doign BOW analysis)
- adjusted width to 128, nominal performance gain
- tok2vec model embed attributed modified to include:
    - "ORTH", "LOWER", "NORM", "PREFIX", "SUFFIX", "SHAPE", "ID"
    - [2000, 2000, 1000, 1000, 1000, 1000]

Also adjusted the width size from 64 to 96, which only resulted in a nominal increase in performance.

Training was tested on training datasets of 100, 500, 1000 and finally, 5000.

In [ ]:
!python -m spacy evaluate ./models/textCatEnsemble/model-best ./data/dev.spacy --output ./evaluate/model-textCatEnsemble-metrics.json

In [ ]:
!python -m spacy debug data ./config/config-TextCatEnsemble.cfg 